In [1]:
import pandas as pd

pd.set_option('display.max_seq_items', None)

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'NanumGothic' # matplotlib 한글깨짐 해결
mpl.rcParams['axes.unicode_minus'] = False

### 전처리

In [3]:
import pandas as pd
file_path = "final_data_rebounds.csv"
df_origin = pd.read_csv(file_path)

In [8]:
df.columns

Index(['누적관객수', '총매출액', '상영일수', '관객수_mean', '관객수_std', '관객수_min', '관객수_max',
       '관객수_first', '관객수_last', '관객수_sum', '관객수_volatility', '관객수_slope',
       '관객수_peak_offset', '관객수_peak_value', '관객수_early_peak', '관객수_sum_after',
       '스크린수_mean', '스크린수_std', '스크린수_min', '스크린수_max', '스크린수_first',
       '스크린수_last', '스크린수_sum', '스크린수_volatility', '스크린수_slope',
       '스크린수_peak_offset', '스크린수_peak_value', '스크린수_early_peak',
       '스크린수_sum_after', '상영횟수_mean', '상영횟수_std', '상영횟수_min', '상영횟수_max',
       '상영횟수_first', '상영횟수_last', '상영횟수_sum', '상영횟수_volatility', '상영횟수_slope',
       '상영횟수_peak_offset', '상영횟수_peak_value', '상영횟수_early_peak',
       '상영횟수_sum_after', '장르_상영일수_점수', '국적_상영일수_점수', '배우_점수', '배우_영화수_점수',
       '배우_매출액_점수', '감독_점수', '감독_영화수_점수', '감독_매출액_점수', '배급사_점수', '배급사_영화수_점수',
       '배급사_매출액_점수', '흥행안정성지수_10일', '장르_SF', '장르_가족', '장르_공연', '장르_공연실황',
       '장르_공포', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스',
       '장르_모험', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사

In [6]:
cols_to_drop = ['영화명', '개봉일', '장르', '국적', '감독', '배급사', '대표국적', '등급', '제작사', '배우','조회일', '순위', '매출액', '매출액(점유율)', '매출액증감',
       '매출액증감율(전일대비)', '관객수', '관객수증감(전일대비)', '관객수증감율(전일대비)', '스크린수', '상영횟수', 
       '흥행안정성지수_3일', '재반등횟수', '3일뒤_재반등횟수', '7일뒤_재반등횟수', '10일뒤_재반등횟수', '반감기',       ]
df = df_origin.drop(columns=cols_to_drop)

In [9]:
df.fillna(df.mean(), inplace=True) # 결측값은 평균으로 채움

### lsa 적용

In [10]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# 1) 원-핫 컬럼 식별
genre_cols   = [c for c in df.columns if c.startswith('장르_') and not c.endswith('_점수')]
country_cols = [c for c in df.columns if c.startswith('국적_') and not c.endswith('_점수')]
onehot_cols  = genre_cols + country_cols

# 2) 원-핫 데이터 선택 및 스케일링
X_onehot = df[onehot_cols].values
scaler = StandardScaler(with_mean=False)  # 희소 행렬에선 with_mean=False
X_scaled = scaler.fit_transform(X_onehot)

# 3) Truncated SVD로 누적 분산 70%를 설명하는 컴포넌트 개수 찾기
n_features = X_scaled.shape[1]
svd_full = TruncatedSVD(n_components=n_features-1, random_state=42)
svd_full.fit(X_scaled)
cum_var = np.cumsum(svd_full.explained_variance_ratio_)
n_components_70 = np.argmax(cum_var >= 0.70) + 1
print(f"70% 분산을 설명하는 컴포넌트 개수: {n_components_70}")

# 4) 실제 차원 축소
svd = TruncatedSVD(n_components=n_components_70, random_state=42)
X_reduced = svd.fit_transform(X_scaled)

# 5) 압축 피쳐를 DataFrame으로 변환 및 기존 데이터와 결합
reduced_cols = [f'LSA_GC_{i+1}' for i in range(n_components_70)]
df_reduced = pd.DataFrame(X_reduced, columns=reduced_cols, index=df.index)

# 원핫 컬럼 제거 후 병합
df_final = pd.concat([
    df.drop(columns=onehot_cols),
    df_reduced
], axis=1)

print(f"원본 컬럼 수: {len(df.columns)} → 압축 후 컬럼 수: {len(df_final.columns)}")
print(f"총 설명 분산 비율: {svd.explained_variance_ratio_.sum():.4f}")

70% 분산을 설명하는 컴포넌트 개수: 27
원본 컬럼 수: 98 → 압축 후 컬럼 수: 81
총 설명 분산 비율: 0.7113


In [12]:
components = pd.DataFrame(
    svd.components_,
    index=reduced_cols,
    columns=onehot_cols
)

In [13]:
top_n = 5

# 각 컴포넌트별 상위 N개 피처를 dict 형태로 추출
top_dict = {
    comp: components.loc[comp]
                       .nlargest(top_n)   # (값이 큰 순서대로)
                       .index
                       .tolist()
    for comp in components.index
}

# dict → DataFrame 변환 (행: 컴포넌트, 열: Top_1~Top_N)
df_top_features = pd.DataFrame.from_dict(
    top_dict,
    orient='index',
    columns=[f"Top_{i+1}" for i in range(top_n)]
)

# 결과 출력
print(df_top_features)

                 Top_1       Top_2     Top_3       Top_4       Top_5
LSA_GC_1        장르_드라마       국적_미국     국적_한국       장르_액션      장르_스릴러
LSA_GC_2         국적_한국       장르_기타  국적_Other  장르_성인물(에로)    장르_다큐멘터리
LSA_GC_3        국적_프랑스     국적_이탈리아    장르_드라마      국적_스위스       국적_독일
LSA_GC_4        장르_스릴러       장르_기타   장르_미스터리   장르_공포(호러)       장르_범죄
LSA_GC_5      국적_Other       장르_기타  장르_애니메이션     장르_어드벤처       장르_액션
LSA_GC_6         장르_공연      장르_뮤지컬     국적_독일       국적_영국       국적_한국
LSA_GC_7         장르_공연   장르_공포(호러)  장르_애니메이션       국적_일본       국적_독일
LSA_GC_8      국적_Other      장르_뮤지컬     장르_가족      장르_코미디      장르_드라마
LSA_GC_9        장르_코미디      장르_판타지   국적_이탈리아      국적_프랑스       장르_가족
LSA_GC_10     장르_다큐멘터리      국적_스웨덴   국적_네덜란드       장르_가족      국적_스페인
LSA_GC_11        장르_범죄       장르_공연  장르_애니메이션      장르_뮤지컬     장르_공연실황
LSA_GC_12  장르_서부극(웨스턴)      국적_스페인     국적_미국       국적_일본       장르_기타
LSA_GC_13        장르_사극       국적_중국   국적_포르투갈       장르_전쟁   장르_공포(호러)
LSA_GC_14       국적_폴란드       장르_범죄